In [20]:
#stripping sentences of puncutation
def parse_sentence(sentence,punt):
    for x in range(len(sentence)):
        temp = sentence[x]
        sentence[x] = [y for y in temp.rstrip().split(" ") if y not in punt]
    return sentence


In [21]:
#calculating distinct words
#along with their counts
def tally_words(list_of_sentence):
    words = {}
    for x in list_of_sentence:
        for y in range(len(x)):
            if x[y] not in words:
                words[x[y]] = 1
            else:
                temp = words.get(x[y])
                words[x[y]] = temp + 1
    return words

In [22]:
#creating bigrams
def create_bigram(words):
    bi = []
    for x in words:
        for y in range(len(x)-1):
            bi.append((x[y],x[y+1]))
    return bi
    

In [23]:

#totalling bigrams
#counts of bigrams not smooth
def count_bigrams(bi):
    bi_counts = {}
    for x in bi:
        if x not in bi_counts:
            bi_counts[x] = bi.count(x)
    return bi_counts

In [24]:
#creating probabilites
#probabiltes not smooth
def prob_bi(bi,words):
    bi_probs = {}
    for x in bi:
        bi_probs[x] = bi[x] / words[x[0]]
    return bi_probs

In [25]:
#add one smoothing
# count of (bigram + 1) * (number of first)/(number of first + distinct word total)
def add_one_smooth_count(bi,words):
    bigram_smooth = {}
    for x in bi:
        bigram_smooth[x] = (bi[x] + 1) * (words[x[0]]/(words[x[0]] + len(words)))
    return bigram_smooth

In [26]:
#add one smoothing probs
# count of (bigram + 1) /(number of first + distinct word total)
def add_one_smooth_prob(bi,words):
    bigram_smooth_probs = {}
    for x in bi:
        bigram_smooth_probs[x] = (bi[x] + 1) / (words[x[0]] + len(words) )
    return bigram_smooth_probs

In [27]:
import pandas as pd
def test_sentence_counts(words,bi):
    count_DF = pd.DataFrame(columns= list(dict.fromkeys(words)), index = list(dict.fromkeys(words)))
    for row in count_DF.index:
        for column in count_DF.columns:
            temp = 0
            if ((row,column) in bi):
                temp = bi[(row,column)]
            count_DF.loc[row][column] = temp
    return count_DF

def test_sentence_probs(words,bi):
    prob_DF = pd.DataFrame(columns= list(dict.fromkeys(words)), index = list(dict.fromkeys(words)))
    for row in prob_DF.index:
        for column in prob_DF.columns:
            temp = 0
            if ((row,column) in bi):
                temp = bi[(row,column)]
            prob_DF.loc[row][column] = temp
    return prob_DF

def test_sentence_counts_one(words,bi,counts):
    count_DF = pd.DataFrame(columns= list(dict.fromkeys(words)), index = list(dict.fromkeys(words)))
    for row in count_DF.index:
        for column in count_DF.columns:
            temp = (1) * (counts[row]/(counts[row] + len(counts)))
            if ((row,column) in bi):
                temp = bi[(row,column)]
            count_DF.loc[row][column] = temp
    return count_DF

def test_sentence_probs_one(words,bi,counts):
    count_DF = pd.DataFrame(columns= list(dict.fromkeys(words)), index = list(dict.fromkeys(words)))
    for row in count_DF.index:
        for column in count_DF.columns:
            temp = (1) / (counts[row] + len(counts) )
            if ((row,column) in bi):
                temp = bi[(row,column)]
            count_DF.loc[row][column] = temp
    return count_DF


In [28]:
def calc_sentence_probs(df,num,sentence):
   prob = 1/num
   for x in range(1,(len(sentence))):
      prob = prob * df.at[sentence[x-1],sentence[x]]
   return prob

In [31]:

import sys
def main(corpus="corpus.txt",smoothing=1):
    sentence = []
    puncutation = ['.' , '[', ']', '{', '}', '?', '!',':',';', ',', '\'','-','(', ')','\n']
    test_sentence_one = ["mark", "antony" , "heere", "take", "you" ,"caesars", "body" , "you", "shall", "not", "come", "to", "them", "poet"]
    test_sentence_two = ["no", "sir" , "there", "are", "no" ,"comets", "seen" , "the", "heauens", "speede", "thee", "in", "thine", "enterprize"]
    #reading in corpus
    #if len(sys.argv) > 1:
        #if (sys.argv[1].isnumeric()) and len(sys.argv) == 2:
            #smoothing = sys.argv[1]
        #else:
            #corpus = sys.argv[1]
        #if len(sys.argv) > 2:
            #smoothing = sys.argv[2]


    with open(corpus) as my_file:
        next_line = my_file.readline()
        while next_line:
            sentence.append(next_line)
            next_line = my_file.readline()
    
    sentence = parse_sentence(sentence,puncutation)
    distinct_words = tally_words(sentence)
    bigrams = create_bigram(sentence)
    num_words = len(distinct_words)

    bigram_counts = count_bigrams(bigrams)
    bigram_probs = prob_bi(bigram_counts,distinct_words)
    
    bigram_counts_smooth = add_one_smooth_count(bigram_counts,distinct_words)
    bigram_probs_smooth = add_one_smooth_prob(bigram_counts,distinct_words)

    if smoothing == 1:
        count_one = test_sentence_counts_one(test_sentence_one,bigram_counts_smooth,distinct_words)
        prob_one = test_sentence_probs_one(test_sentence_one,bigram_probs_smooth,distinct_words)
        count_two = test_sentence_counts_one(test_sentence_two,bigram_counts_smooth,distinct_words)
        prob_two = test_sentence_probs_one(test_sentence_two,bigram_probs_smooth,distinct_words)
        print("Sentence one counts table with smoothing")
        print(count_one)
        print("Sentence one probability table with smoothing")
        print(prob_one)
        print("Sentence Probability = " + str(calc_sentence_probs(prob_one,num_words,test_sentence_one)))
        print("----------------------------------------------------------------------")
        print("Sentence two counts table with smoothing")
        print(count_two)
        print("Sentence two probability table with smoothing")
        print(prob_two)
        print("Sentence Probability = " + str(calc_sentence_probs(prob_two,num_words,test_sentence_two)))
    else:
        count_one = test_sentence_counts(test_sentence_one,bigram_counts)
        prob_one = test_sentence_probs(test_sentence_one,bigram_probs)
        count_two = test_sentence_counts(test_sentence_two,bigram_counts)
        prob_two = test_sentence_probs(test_sentence_two,bigram_probs)
        print("Sentence one counts table no smoothing")
        print(count_one)
        print("Sentence one probability table no smoothing")
        print(prob_one)
        print("Sentence Probability = " + str(calc_sentence_probs(prob_one,num_words,test_sentence_one)))
        print("----------------------------------------------------------------------")
        print("Sentence two counts table no smoothing")
        print(count_two)
        print("Sentence two probability table no smoothing")
        print(prob_two)
        print("Sentence Probability = " + str(calc_sentence_probs(prob_two,num_words,test_sentence_two)))

In [32]:
if __name__ == "__main__":
    main()

Sentence one counts table with smoothing
             mark    antony     heere      take       you   caesars      body  \
mark     0.004614  0.064601  0.004614  0.004614  0.004614  0.004614  0.004614   
antony   0.024233  0.024233  0.048465  0.024233  0.048465  0.024233  0.024233   
heere    0.019162  0.019162  0.038324  0.057486  0.019162  0.019162  0.019162   
take     0.007558  0.007558  0.007558  0.007558  0.015117  0.007558  0.007558   
you      0.114629  0.229258  0.458517  0.114629  0.343887  0.229258  0.114629   
caesars  0.012426  0.012426  0.012426  0.012426  0.012426  0.012426  0.037279   
body     0.004286  0.004286  0.004286  0.004286  0.008572  0.004286  0.004286   
shall    0.039746  0.039746  0.039746  0.079491  0.079491  0.039746  0.039746   
not      0.078425  0.078425  0.078425  0.156851  0.235276  0.078425  0.078425   
come     0.023917  0.071752  0.023917  0.023917  0.023917  0.023917  0.023917   
to       0.128679  0.128679  0.128679  0.257357  2.058857  0.643393 